# Meteorology-SnowDegreeDay coupling

**Goal:** Determine why a coupled `Meteorology-SnowDegreeDay` simulation fails, when each component runs to completion in stand-alone mode.

Start with an import and some magic:

In [ ]:
%matplotlib inline
import numpy as np

Import the Babel-wrapped `Meteorology` and `SnowDegreeDay` components and create instances:

In [ ]:
from cmt.components import Meteorology, SnowDegreeDay
met, sno = Meteorology(), SnowDegreeDay()

## Demonstrate that the models run independently

Initialize the components with cfg files that, for simplicity, use the same time step and run duration:

In [ ]:
met.initialize('./input/meteorology-2.cfg')
sno.initialize('./input/snow_degree_day-1.cfg')

Get the model time step (which has been broadcast over the grid):

In [ ]:
time_step = met.get_value('model__time_step')
time_step = time_step.max()
print 'Time step = {} s'.format(time_step)

Advance both models to the end, saving the model time, precipitation flux, and snowmelt flux values at each step:

In [ ]:
time, precip, melt = [], [], []
until = time_step
while met.get_current_time() < met.get_end_time():
    met.update(until)
    time.append(met.get_current_time())
    precip.append(met.get_value('atmosphere_water__precipitation_leq-volume_flux'))
    melt.append(sno.get_value('snowpack__melt_volume_flux'))
    until += time_step

Check the values:

In [ ]:
time

In [ ]:
precip

In [ ]:
melt

Finalize the components:

In [ ]:
met.finalize(), sno.finalize()

**Result:** The models run independently.

## Couple the models

Initialize the components with cfg files that, for simplicity, use the same time step and run duration:

In [ ]:
met.initialize('./input/meteorology-2.cfg')
sno.initialize('./input/snow_degree_day-1.cfg')

Get the model time step (which has been broadcast over the grid):

In [ ]:
time_step = met.get_value('model__time_step')
time_step = time_step.max()
print 'Time step = {} s'.format(time_step)

Advance `SnowDegreeDay` by one time step, getting values to pass to `Meteorology`:

In [ ]:
sno.update(time_step)
rho_snow = sno.get_value('snowpack__z_mean_of_mass-per-volume_density')
h_snow = sno.get_value('snowpack__depth')
h_swe = sno.get_value('snowpack__liquid-equivalent_depth')
SM = sno.get_value('snowpack__melt_volume_flux')

Pass variables from `SnowDegreeDay` into `Meteorology`, then advance `Meteorology`:

In [ ]:
met.set_value('snowpack__z_mean_of_mass-per-volume_density', rho_snow)
met.set_value('snowpack__depth', h_snow)  # this is the only variable that fails
met.set_value('snowpack__liquid-equivalent_depth', h_swe)
met.set_value('snowpack__melt_volume_flux', SM)
met.update(time_step)

Note the differing shapes of the variables in the statement that fails:

```
Qn_SW.shape = (44, 29)
Qn_LW.shape = ()
Qh.shape = (1276,)
Qe.shape = (1276,)
Qa.shape = ()
Qc.shape = ()
```

Finalize the components:

In [ ]:
met.finalize(), sno.finalize()